In [1]:
import gradio as gr
import numpy as np
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Set device to CUDA if available, else CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the pre-trained model and tokenizer
model_path = "C:\\Users\\thema\\Desktop\\Content_2\\USD\\Natural Language Processing\\Final Project\\Model"
model = GPT2LMHeadModel.from_pretrained(model_path).to(device)
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-medium')
tokenizer.pad_token = tokenizer.eos_token


class Chat:
    def __init__(self, chatbot, max_response_len=20):
        self.chatbot = chatbot
        self.messages = []
        self.max_response_len = max_response_len
        self.temperature = 0.2  # Default temperature
        self.top_p = 0.3  # Default top_p

    def send(self, text: str):
        response = self.handle_input(text)
        self.messages.append(f"User: {text}")
        self.messages.append(f"Bot: {response}")
        return response

    def set_topic(self, topic: str):
        self.messages.append(f"Topic: {topic}")

    def reset_conversation(self):
        self.messages = []

    def set_mood(self, temperature: float, top_p: float):
        self.temperature = temperature
        self.top_p = top_p

    def handle_input(self, text: str):
        input_type = self.classify_input(text)
        if input_type == "question":
            response = self.chatbot.generate(self.get_context() + f"User: {text}", max_length=self.max_response_len, temperature=self.temperature, top_p=self.top_p)
        elif input_type == "acknowledgment":
            response = self.generate_acknowledgment_response(text)
        else:
            response = self.generate_follow_up(text)
        return response

    def classify_input(self, text: str) -> str:
        if "?" in text or text.lower().startswith(("how", "what", "why", "where", "when", "who")):
            return "question"
        elif text.lower() in ["thanks", "thank you", "okay", "got it", "appreciate it"]:
            return "acknowledgment"
        else:
            return "statement"

    def generate_acknowledgment_response(self, text: str) -> str:
        prompt = self.get_context() + f"User: {text}\n Chatbot: You're welcome! I hope that helps."
        response = self.chatbot.generate(prompt, max_length=self.max_response_len, temperature=self.temperature, top_p=self.top_p)
        return response

    def generate_follow_up(self, text: str) -> str:
        prompt = self.get_context() + f"User: {text}\n Chatbot: That's an interesting point."
        response = self.chatbot.generate(prompt, max_length=self.max_response_len, temperature=self.temperature, top_p=self.top_p)
        return response

    def get_context(self) -> str:
        prompt = '<|endoftext|>'.join(self.messages) + '<|endoftext|>'
        max_tokens = 112
        prompt_tokens = self.chatbot.tokenizer.encode(prompt)
        if len(prompt_tokens) > max_tokens:
            prompt_tokens = prompt_tokens[-max_tokens:]
            prompt = self.chatbot.tokenizer.decode(prompt_tokens)
        return prompt


class ChatBot:
    def __init__(self, model_path: str, device=None):
        if not device:
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.device = device
        self.model = GPT2LMHeadModel.from_pretrained(model_path).to(self.device)
        self.tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-medium')
        self.tokenizer.pad_token = self.tokenizer.eos_token

    def generate(self, text: str, max_length: int = 12, temperature: float = 0.6, top_p: float = 0.9) -> str:
        with torch.no_grad():
            inputs = self.tokenizer(text, return_tensors="pt")
            outputs = self.model.generate(
                inputs.input_ids.to(self.device),
                attention_mask=inputs.attention_mask.to(self.device),
                max_new_tokens=50,
                no_repeat_ngram_size=5,
                early_stopping=True,
                num_beams=2,
                pad_token_id=self.tokenizer.eos_token_id,
                eos_token_id=self.tokenizer.eos_token_id,
                repetition_penalty=10.0,
                temperature=temperature,
                top_p=top_p,
                do_sample=True,
                length_penalty=0.8
            )
            response_outputs = outputs[:, len(inputs['input_ids'][0]):]
            response = self.tokenizer.batch_decode(response_outputs, skip_special_tokens=True)[0]
            return response

    def create_chat(self) -> Chat:
        return Chat(self)




In [2]:
# Initialize the chatbot
chatBot = ChatBot(model_path)
conversation = chatBot.create_chat()


# Define functions for Gradio interface
def chatbot_response(message):
    return conversation.send(message)

def set_topic(topic):
    conversation.set_topic(topic)
    return "Topic set."

def reset_conversation():
    conversation.reset_conversation()
    return "Conversation reset."

def set_mood(temperature, top_p):
    conversation.set_mood(temperature, top_p)
    return f"Mood set to Temperature: {temperature}, Top P: {top_p}"


# Create Gradio interface using Blocks
with gr.Blocks() as demo:
    gr.Markdown("## Customizable Chatbot")
    
    # Main Chat Interface
    with gr.Row():
        chatbot_input = gr.Textbox(label="Type your message", placeholder="Send a message to the chatbot")
        chatbot_output = gr.Textbox(label="Chatbot's Response", interactive=False)
    
    # Chatbot interaction button
    with gr.Row():
        send_button = gr.Button("Send")
    
    send_button.click(chatbot_response, inputs=chatbot_input, outputs=chatbot_output)

    # Controls for setting mood, topic, and resetting conversation
    with gr.Row():
        with gr.Column():
            topic_input = gr.Textbox(label="Set Topic", placeholder="Enter conversation topic")
            topic_button = gr.Button("Set Topic")
            topic_button.click(set_topic, inputs=topic_input, outputs=chatbot_output)
        
        with gr.Column():
            temperature_slider = gr.Slider(minimum=0.1, maximum=1.0, label="Temperature", step=0.1, value=0.6)
            top_p_slider = gr.Slider(minimum=0.1, maximum=1.0, label="Top P", step=0.1, value=0.9)
            mood_button = gr.Button("Set Mood")
            mood_button.click(set_mood, inputs=[temperature_slider, top_p_slider], outputs=chatbot_output)
        
        with gr.Column():
            reset_button = gr.Button("Reset Conversation")
            reset_button.click(reset_conversation, outputs=chatbot_output)

# Launch the Gradio app
demo.launch()


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
